## Optimal offering strategy for large producers based on market price response learning

The following section describes the optimization problem that is is employed by the GENCO to derive her optimal offering strategy. The problem is formulated using a risk-averse two-stage stochastic approach in which the marginal price of the market is explicitly characterized through constraint learning.

### Notation

The notation employed to formulate the stochastic problem is described in this subsection for reference.

Indices and sets:

* $I$: Set of energy blocks, indexed by $i$.
* $T$: Set oh hourly periods within a day, indexed by $t$.
* $\Omega$: Set of stochastic scenarios, indexed by $\omega$.


Decision variables:
* $P^i_{t}$: Price established for block $i$ at time $t$.
* $Q_{t,\omega}^{i}$: Quantity of energy from block $i$ produced at time $t$ and scenario $\omega$.
* $u_{t,\omega}^i$: Binary decision variable indicating whether price from block $i$ at time $t$ and scenario $\omega$ is under the marginal electricity price or not.
* $s_{\omega}$: Auxiliary variables for CVaR formulation.
* $\eta$: Auxiliary variable for CVaR formulation.


Variables:
* $\lambda_{t,\omega} :$ Marginal market price at time $t$ and scenario $\omega$.
* $Q^{ren}_{t} :$ Offered quantity at price zero (from renewable resources) for each time.


Parameters:
* $\beta^i_{t,\omega}$: Coefficient of the 7 different block price predictors for each time at scenario $\omega$, sampled from their distribution.
* $\hat{D}_t$: Value composed by the sum of coefficients multiplied by the rest of variables within the linear model (demand, wind and solar energy forecasting with their respective lags, marginal price lags, and calendar variables), at time $t$.  
* $C_t^i:$ Cost of production for block $i$ at time $t$.
* $\sigma_t^i$: Allowed variability for the block price with respect to the block cost of production at time $t$.
* $\pi_{\omega}:$ Probability assigned to each scenario.
* $\alpha$: Fraction of the distribution to be used in the CVaR calculation.
* $\chi$: Weight assigned to the CVaR against the profit.


### Formulation
\begin{align*}
\underset{P_t^i, Q_{t,\omega}^{i}, u_{t,\omega}^i, \eta, s_{\omega}}{\max} & \quad (1-\chi) \sum_{\omega \in \Omega} \pi_{\omega} \sum_{t \in T} \left[\lambda_{t,\omega} Q_t^{ren} + \sum_{i\in I}  (\lambda_{t,\omega} Q_{t,\omega}^{i} - C_t^i Q_{t,\omega}^{i}) \right] + \chi \left(\eta - \frac{1}{\alpha} \sum_{\omega \in \Omega} \pi_{\omega} s_{\omega} \right)\\
\text{s.t.:}& \\
  &Q_t^{ren} = Q_t^{\text{Max ren}} \quad \forall t \\
  &u_{t,\omega}^i \in \{0,1\} \quad \forall i,t,\omega\\
  &\lambda_{t,\omega} - P_t^i \leq u_{t,\omega}^iM  \quad \forall i,t,\omega\\
  &P_t^i - \lambda_{t,\omega} \leq (1-u_{t,\omega}^i)M  \quad \forall i,t,\omega \\
  &Q_{t,\omega}^{i} = u_{t,\omega}^i Q_t^{\text{Max }i} \quad \forall i,t,\omega \\
  &\lambda_{t,\omega} = \beta_0 + \beta^{ren} Q_t^{ren} + \sum_i \beta^i_{t,\omega} P_t^i + D_t \quad \forall i,t,\omega \\
  &C_t^i - \sigma_t^i \leq P_t^i \leq C_t^i + \sigma_t^i \quad \forall i,t \\
  &0 < P_t^i \leq P_t^{i+1} \quad \forall i,t \\
  &\eta - \sum_t \left[\lambda_{t,\omega} Q_t^{ren} + \sum_i  (\lambda_{t,\omega} Q_{t,\omega}^{i} - C_t^i Q_{t,\omega}^{i}) \right] \leq s_{\omega} \quad \forall \omega \\
  &0 \leq s_{\omega} \quad \forall \omega 
\end{align*}




In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyomo.environ as pyo
solver = pyo.SolverFactory("gurobi")
from __future__ import division 


#### Pyomo model

In [2]:
def opt_block_offer(C, pred_qr, Qmax, reg_sum, coefs_price, M, sigma, n_sc, pi, alfa, chi):
    
    model = pyo.ConcreteModel()
    
    model.I = pyo.RangeSet(1,6)
    model.T = pyo.RangeSet(1,24)
    model.W = pyo.RangeSet(1,n_sc) # scenarios
    
    
    # Variables
    model.u = pyo.Var(model.I, model.T, model.W, within=pyo.Binary) # binary for matching quantities
    model.Q = pyo.Var(model.I, model.T, model.W, within=pyo.NonNegativeReals) # block quantity
    model.Qren = pyo.Var(model.T, within=pyo.NonNegativeReals) # renewable energy quantity
    model.PM = pyo.Var(model.T, model.W, within=pyo.NonNegativeReals) # marginal price
    model.P = pyo.Var(model.I, model.T, within=pyo.NonNegativeReals) # block price
    model.z = pyo.Var(model.I, model.T, model.W, within=pyo.NonNegativeReals) # auxiliary variable to linearize
    
    
    model.Prof = pyo.Var(model.W, within=pyo.NonNegativeReals) # saving profit per scenario
    
    model.eta = pyo.Var(within=pyo.NonNegativeReals) # for cvar
    model.s = pyo.Var(model.W, within=pyo.NonNegativeReals) #for cvar
    model.cvar = pyo.Var(within=pyo.NonNegativeReals) # saving cvar
    
    
    #definition of the objective function
    def obj_expression(model):
        return (1-chi)*sum(pi[w-1] * model.Prof[w] for w in model.W) + chi*model.cvar
    model.OBJ = pyo.Objective(rule=obj_expression, sense=pyo.maximize)
    
    
    #constraint q_ren
    def const1_rule(model,t): 
        return model.Qren[t] == pred_qr[t-1]  
    model.const1 = pyo.Constraint(model.T, rule=const1_rule)
    
    
    #constraints for matching energy
    def const2_rule(model,i,t,w):
        return model.Q[i,t,w] ==  model.u[i,t,w]*Qmax[i-1,t-1] 
    model.const2 = pyo.Constraint(model.I, model.T, model.W, rule=const2_rule)
    
    
    #constraints for matching energy
    def const3a_rule(model,i,t,w):
        return model.PM[t,w] - model.P[i,t] <=  model.u[i,t,w] * M
    model.const3a = pyo.Constraint(model.I, model.T, model.W, rule=const3a_rule)
    def const3b_rule(model,i,t,w):
        return model.P[i,t] - model.PM[t,w] <= (1-model.u[i,t,w]) * M
    model.const3b = pyo.Constraint(model.I, model.T, model.W, rule=const3b_rule)
    
    
    #constraint for marginal price response learning
    def const4_rule(model,t,w):
        price_sum = sum((model.P[i,t] * coefs_price[i-1,w-1]) for i in model.I)
        suma2 = intercep + model.Qren[t] * beta_qren + price_sum
        return model.PM[t,w] == suma2 + reg_sum[t-1]
    model.const4 = pyo.Constraint(model.T, model.W, rule=const4_rule)
    
    
    #constrain for price-cost relationship
    def const5a_rule(model,i,t):
        return model.P[i,t] <= C[i-1,t-1] + sigma[i-1,t-1]
    model.const5a = pyo.Constraint(model.I, model.T, rule=const5a_rule)
    
    def const5b_rule(model,i,t):
        return model.P[i,t] >= C[i-1,t-1] - sigma[i-1,t-1]
    model.const5b = pyo.Constraint(model.I, model.T, rule=const5b_rule)
    
    
    #costraint for differences between block prices
    def const6_rule(model,i,t):
        if i==6:
            return pyo.Constraint.Skip
        else:
            return model.P[i,t] <= model.P[i+1,t]
    model.const6 = pyo.Constraint(model.I, model.T, rule=const6_rule)
    
    
    
    #to linearize
    def const9a_rule(model,i,t,w):
        return model.z[i,t,w] <=  model.u[i,t,w] * M
    model.const9a = pyo.Constraint(model.I, model.T, model.W, rule=const9a_rule)
    
    def const9b_rule(model,i,t,w):
        return model.z[i,t,w] >=  model.PM[t,w] - (1 - model.u[i,t,w]) * M
    model.const9b = pyo.Constraint(model.I, model.T, model.W, rule=const9b_rule)
    
    def const9c_rule(model,i,t,w):
        return model.z[i,t,w] <=  model.PM[t,w]
    model.const9c = pyo.Constraint(model.I, model.T, model.W, rule=const9c_rule)
    
    
    #saving profit
    def const10_rule(model,w):
        return model.Prof[w] ==  sum(model.PM[t,w]*model.Qren[t] + sum((model.z[i,t,w]*Qmax[i-1,t-1] - model.Q[i,t,w]*C[i-1,t-1]) for i in model.I) for t in model.T)
    model.const10 = pyo.Constraint(model.W, rule=const10_rule)
    
    #saving cvar
    def const11_rule(model):
        return model.cvar == (model.eta - 1/alfa * sum(pi[w-1] * model.s[w] for w in model.W))
    model.const11 = pyo.Constraint(rule=const11_rule)
    
    #constraints for cvar
    def const12_rule(model, w):
        return model.eta - model.Prof[w] <= model.s[w]
    model.const12 = pyo.Constraint(model.W, rule=const12_rule)
    
    
    return model




#### Data for marginal price learning and scenario generation

In [3]:
dataset_full = pd.read_csv('dataset_price_learning.csv')
dataset_full['fecha2'] = pd.to_datetime(dataset_full['fecha2'])
dataset_full.set_index('fecha2', inplace=True)

dataset_train = dataset_full[dataset_full.index < pd.to_datetime('2019-06-01')]
dataset_test = dataset_full[dataset_full.index >= pd.to_datetime('2019-06-01')]

X_train = dataset_train.drop(['precio'], axis=1)
y_train = dataset_train.precio

X_test = dataset_test.drop(['precio'], axis=1)
y_test = dataset_test.precio

dates = X_test.index

In [4]:
coefs_total = pd.read_csv('coef_distribution.csv', index_col=0)

intercep = coefs_total.means[0]
beta_qren = coefs_total.means[1]
beta_prices = coefs_total.means[2:8].values.tolist()
std_prices= coefs_total.stds[2:8].values.tolist()

#### Function for regression calculation outside decision variables

In [5]:
def calc_reg(X, coefs_total, per2):
    X_set = X.iloc[0+per2:24+per2].drop(['q_ren','p1','p2','p3','p4','p5','p6','q1','q2','q3','q4','q5','q6'], axis=1)
    reg_sum = []
    t = 0
    for i in X_set.index:
        X_value = X_set[X_set.index == i]
        exo_sum = (X_value * coefs_total.means[8:].values).sum(axis=1)
        reg_sum = np.append(reg_sum, exo_sum)
        t = t+1
    
    return reg_sum


#### Optimization

In [6]:
M = 10000

Profit_opt = []
Qren_opt = []
PM_opt = []
P1_opt = []
P2_opt = []
P3_opt = []
P4_opt = []
P5_opt = []
P6_opt = []


vals_q = np.ndarray(shape=(6,24))
Q_opt = np.ndarray(shape=(6,24))

Q1_opt = []
Q2_opt = []
Q3_opt = []
Q4_opt = []
Q5_opt = []
Q6_opt = []

cvar = []

pm_df = pd.DataFrame()
profit_df = pd.DataFrame()

np.random.seed(0)
coefs_price = np.array([np.random.normal(m, s, 200) for m, s in zip(beta_prices, std_prices)])


for per in range(30):
    print('Setting optimal block prices for day', per)
    per2 = 24*per
    
    pi_arr = [1/200 for i in range(200)]
    
    X_test_tmp = X_test.iloc[0+per2:24+per2]
    
    # we assume knowledge about the quantity we can offer and its cost
    
    c1 = np.array(X_test_tmp.p1)
    c2 = np.array(X_test_tmp.p2)
    c3 = np.array(X_test_tmp.p3)
    c4 = np.array(X_test_tmp.p4)
    c5 = np.array(X_test_tmp.p5)
    c6 = np.array(X_test_tmp.p6)
    
    q1 = np.array(X_test_tmp.q1)
    q2 = np.array(X_test_tmp.q2)
    q3 = np.array(X_test_tmp.q3)
    q4 = np.array(X_test_tmp.q4)
    q5 = np.array(X_test_tmp.q5)
    q6 = np.array(X_test_tmp.q6)
    
    pred_qr = np.array(X_test_tmp.q_ren)
    
    C = np.concatenate((c1,c2,c3,c4,c5,c6)).reshape(6,24)
    Qmax = np.concatenate((q1,q2,q3,q4,q5,q6)).reshape(6,24)
    
    s = 0.1
    sigma = np.concatenate((s*c1,s*c2,s*c3,s*c4,0*c5,0*c6)).reshape(6,24)   # setting sigma, pricing flexiblity outside from costs
    
    reg_sum = calc_reg(X_test, coefs_total, per2)
    
    model = opt_block_offer(C, pred_qr, Qmax, reg_sum, coefs_price, M, sigma, n_sc=200, pi=pi_arr, alfa=0.1, chi=0.001)
    solver.options["threads"] = 16
    solver.options["MIPGap"] = 1e-3
    results = solver.solve(model,tee=False)
    
    
    profit_df = profit_df.append([pyo.value(model.Prof[:])])
    
    cvar = np.append(cvar, pyo.value(model.cvar))
    
    Qren_opt = np.append(Qren_opt, [pyo.value(model.Qren[t]) for t in model.T])
    
    PM_opt = np.append(PM_opt, [np.mean(pyo.value(model.PM[t,:])) for t in model.T])
    
    for t in model.T:
        vals_pm = pyo.value(model.PM[t,:])
        pm_df = pm_df.append([vals_pm])
    
    P1_opt = np.append(P1_opt, [pyo.value(model.P[i,:]) for i in model.I][0])  # saving block prices
    P2_opt = np.append(P2_opt, [pyo.value(model.P[i,:]) for i in model.I][1])
    P3_opt = np.append(P3_opt, [pyo.value(model.P[i,:]) for i in model.I][2])
    P4_opt = np.append(P4_opt, [pyo.value(model.P[i,:]) for i in model.I][3])
    P5_opt = np.append(P5_opt, [pyo.value(model.P[i,:]) for i in model.I][4])
    P6_opt = np.append(P6_opt, [pyo.value(model.P[i,:]) for i in model.I][5])

    
    for i in model.I:
        for t in model.T:
            vals = np.mean(pyo.value(model.Q[i,t,:]))
            vals_q[i-1,t-1] = vals
    
    Q_opt = np.append(Q_opt, vals_q, axis=1)

Q1_opt = np.delete(Q_opt[0,:], slice(24))  # saving matching energy
Q2_opt = np.delete(Q_opt[1,:], slice(24))
Q3_opt = np.delete(Q_opt[2,:], slice(24))
Q4_opt = np.delete(Q_opt[3,:], slice(24))
Q5_opt = np.delete(Q_opt[4,:], slice(24))
Q6_opt = np.delete(Q_opt[5,:], slice(24))


pm_df.set_index(dates, inplace=True)
profit_df.set_index(pd.date_range('2019-06-01','2019-06-30').date, inplace=True)

Setting optimal block prices for day 0
Setting optimal block prices for day 1
Setting optimal block prices for day 2
Setting optimal block prices for day 3
Setting optimal block prices for day 4
Setting optimal block prices for day 5
Setting optimal block prices for day 6
Setting optimal block prices for day 7
Setting optimal block prices for day 8
Setting optimal block prices for day 9
Setting optimal block prices for day 10
Setting optimal block prices for day 11
Setting optimal block prices for day 12
Setting optimal block prices for day 13
Setting optimal block prices for day 14
Setting optimal block prices for day 15
Setting optimal block prices for day 16
Setting optimal block prices for day 17
Setting optimal block prices for day 18
Setting optimal block prices for day 19
Setting optimal block prices for day 20
Setting optimal block prices for day 21
Setting optimal block prices for day 22
Setting optimal block prices for day 23
Setting optimal block prices for day 24
Setting op

In [7]:
profit_df.to_csv('stoch_profit_chi0_s1.csv')
pd.DataFrame([P1_opt,P2_opt,P3_opt,P4_opt], index=['p1','p2','p3','p4']).to_csv('sto_prices_chi0_s1.csv')
Q_prod = Qren_opt + Q1_opt + Q2_opt + Q3_opt + Q4_opt + Q5_opt + Q6_opt

In [8]:
print('Mean produced energy per hour:', np.mean(Q_prod))

Mean produced energy per hour: 4009.5575315257897


In [9]:
print('Mean price for block 1:', P1_opt.mean())
print('Mean price for block 2:', P2_opt.mean())
print('Mean price for block 3:', P3_opt.mean())
print('Mean price for block 4:', P4_opt.mean())

Mean price for block 1: 34.56769102572764
Mean price for block 2: 54.48071547325392
Mean price for block 3: 66.69739073823726
Mean price for block 4: 84.00311858123112


In [20]:
print('Mean expected hourly marginal price:',PM_opt.mean())

Mean expected hourly marginal price: 44.50612868606187


### Out-of-sample validation

In [10]:
df_market_blocks = pd.read_csv('market_blocks.csv')
df_market_blocks.drop('Unnamed: 0', axis=1, inplace=True)
df_market_blocks = df_market_blocks[df_market_blocks['GENCO'] == 'No']
df_market_blocks[['fecha', 'periodo']] = df_market_blocks['fecha2'].str.split(' ', 1, expand=True)
df_market_blocks['periodo'] = df_market_blocks.periodo.astype('float')
df_market_blocks['fecha'] = pd.to_datetime(df_market_blocks['fecha'])
df_market_blocks = df_market_blocks[df_market_blocks['fecha'] >= pd.to_datetime('2019-06-01')]

In [11]:
data_df = pd.DataFrame(columns = ['fecha2', 'q_ren', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'q1', 'q2', 'q3', 'q4', 'q5', 'q6'])

for fech in df_market_blocks.fecha2.unique():
    block_tmp = df_market_blocks[df_market_blocks.fecha2 == fech]
    block_tmp.sort_values(['C'], inplace=True)
    
    q_ren = block_tmp.quant_block.values[0]
    quant = block_tmp.quant_block.values[1:]
    prices = block_tmp.C.values[1:]
    
    data_df_tmp = pd.DataFrame([np.hstack((q_ren, prices, quant))], columns = ['q_ren', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'q1', 'q2', 'q3', 'q4', 'q5', 'q6'])
    data_df_tmp['fecha2'] = fech
    data_df = data_df.append(data_df_tmp)

data_df[['fecha', 'periodo']] = data_df['fecha2'].str.split(' ', 1, expand=True)
data_df['periodo'] = data_df.periodo.astype('float')
data_df['fecha'] = pd.to_datetime(data_df['fecha'])

C:\Users\antalcan\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [12]:
demand = pd.read_csv('demanda_prev.csv', sep = ';', engine = 'python')
demand = demand[['value','datetime']]
demand['fecha'] = demand['datetime'].apply(lambda x: x.split('T')[0])
demand['fecha'] = pd.to_datetime(demand['fecha'])
demand['periodo'] = demand.groupby('fecha').cumcount()
demand['periodo'] = demand['periodo'] + 1
demand = demand[demand['fecha'].dt.year <= 2019]
demand = demand[['fecha','periodo','value']]
demand.rename({'value': 'demand'}, axis=1, inplace=True)
demand['fecha2'] = demand.fecha.astype('str') + ' ' + demand.periodo.astype('str') + '.0'

In [13]:
dif_matching_offers = pd.read_csv('dif_matching_offers.csv')
dif_matching_offers['fecha'] = pd.to_datetime(dif_matching_offers['fecha'])
difs = dif_matching_offers.groupby(['periodo']).median()

In [14]:
x_new = np.linspace(0, 60000, 60001)
df_test = pd.DataFrame(columns = ['fecha', 'periodo','pre_oos', 'pre_estima', 'profit'])

i = 0
for fech in profit_df.index:
    # we join the optimal GENCO pricing with the rest of market blocks, and compute a marginal price approximaiton
    X_test_tmp = X_test[X_test.index.date == fech]
    
    c1 = np.array(X_test_tmp.p1)
    c2 = np.array(X_test_tmp.p2)
    c3 = np.array(X_test_tmp.p3)
    c4 = np.array(X_test_tmp.p4)
    c5 = np.array(X_test_tmp.p5)
    c6 = np.array(X_test_tmp.p6)
    
    q1 = np.array(X_test_tmp.q1)
    q2 = np.array(X_test_tmp.q2)
    q3 = np.array(X_test_tmp.q3)
    q4 = np.array(X_test_tmp.q4)
    q5 = np.array(X_test_tmp.q5)
    q6 = np.array(X_test_tmp.q6)
    
    C = np.concatenate((c1,c2,c3,c4,c5,c6)).reshape(6,24)
    Qmax = np.concatenate((q1,q2,q3,q4,q5,q6)).reshape(6,24)
    C_new = np.concatenate((np.array([0 for i in range(24)]).reshape(1,24), C))
    
    t=0
    for per in range(1,25):
        df_market_blocks2 = data_df.iloc[i]
        df_market_blocks2_array = np.array([[df_market_blocks2.q_ren, 0],
                            [df_market_blocks2.q1, df_market_blocks2.p1],
                            [df_market_blocks2.q2, df_market_blocks2.p2],
                            [df_market_blocks2.q3, df_market_blocks2.p3],
                            [df_market_blocks2.q4, df_market_blocks2.p4],
                            [df_market_blocks2.q5, df_market_blocks2.p5],
                            [df_market_blocks2.q6, df_market_blocks2.p6]])
        
        df_market_blocks3 = pd.DataFrame(df_market_blocks2_array, columns=['quant', 'prices'])
        
        df_genco = np.array([[Qren_opt[i], 0],
                            [Qmax[0,t], P1_opt[i]],
                            [Qmax[1,t], P2_opt[i]],
                            [Qmax[2,t], P3_opt[i]],
                            [Qmax[3,t], P4_opt[i]],
                            [Qmax[4,t], P5_opt[i]],
                            [Qmax[5,t], P6_opt[i]]])
        df_genco = pd.DataFrame(df_genco)
        df_genco.columns = df_market_blocks3.columns
        
        dif_tmp = difs[difs.index == per]['dif'].values
        
        df_oferta_prueba = pd.concat([df_market_blocks3, df_genco], ignore_index=True).sort_values(['prices'])
        df_oferta_prueba['quant_cum'] = df_oferta_prueba['quant'].cumsum()
        df_oferta_prueba['quant_cum'] = df_oferta_prueba['quant_cum'] - dif_tmp
        
        df_compra_test_prueba = demand[(demand['periodo'] == per) & (demand['fecha'] == pd.to_datetime(fech))].values[0][2]
        
        block_tmp = df_oferta_prueba[df_oferta_prueba.quant_cum >= df_compra_test_prueba]
        
        price_cut = block_tmp.prices.min()
        pre_estimated = PM_opt[i]
        
        df_ende_prueba_mat = df_genco[df_genco['prices'] <= price_cut]['quant']
        C_mat = C_new[0:len(df_ende_prueba_mat), t]
        prof = sum(df_ende_prueba_mat * price_cut) - sum(df_ende_prueba_mat * C_mat)
        
        df_test = df_test.append({'fecha':fech, 'periodo':per, 'pre_oos':price_cut, 'pre_estima':pre_estimated, 'profit':prof}, ignore_index=True)
        
        i += 1
        t += 1
        

In [15]:
df_test.to_csv('oos_pre_ch0_s1.csv')
df_test_profit = pd.DataFrame(df_test.groupby('fecha')['profit'].sum())
df_test_profit.to_csv('oos_profit_chi0_s1.csv')

In [16]:
print('Out-of-sample mean profit:', df_test_profit.profit.mean())
print('Out-of-sample Q1 profit:', df_test_profit.profit.quantile(0.25))
print('Out-of-sample Q3 profit:', df_test_profit.profit.quantile(0.75))
print('Out-of-sample profit var:', df_test_profit.profit.std())
print('Mean hourly marginal price oos:', df_test.pre_oos.mean())


Out-of-sample mean profit: 3573730.9556290447
Out-of-sample Q1 profit: 3302773.3207660713
Out-of-sample Q3 profit: 4042777.5104507566
Out-of-sample profit var: 664289.2210678256
Mean hourly marginal price oos: 43.93371179739847
